In [1]:
# !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
# !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
# !apt-get update -qq 2>&1 > /dev/null
# !apt-get -y install -qq google-drive-ocamlfuse fuse
# from google.colab import auth
# auth.authenticate_user()
# from oauth2client.client import GoogleCredentials
# creds = GoogleCredentials.get_application_default()
# import getpass
# !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
# vcode = getpass.getpass()
# !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 130812 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.3-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
# !mkdir -p drive
# !google-drive-ocamlfuse drive  -o nonempty

In [0]:
# import os
# os.chdir('drive')

In [5]:
# !ls

'Bello Zone'			 'MLIM7349 Case Study.pdf'
 Competitions			 'Model Merge1.ipynb'
 CRF++-0.58.tar.gz（副本）	 'New Recording.1.m4a'
'Data agreement-LIU Ruilun.pdf'  'New Recording.mp3'
 Dissertation			 'Sign up.pdf'
 lgbm_feature_selection.ipynb	  Untitled0.ipynb


In [1]:
# mainpath=r'./Competitions/KDD-Cup-2019-CAMMTR/Data/data_set_phase1/'
mainpath = r'../Data/data_set_phase1/'

In [8]:
# !ls ./Competitions/KDD-Cup-2019-CAMMTR/Data/data_set_phase1/

profiles.csv	test_queries.csv  train_plans.csv
test_plans.csv	train_clicks.csv  train_queries.csv


In [2]:
import pandas as pd
import numpy as np

In [3]:
train_queries = pd.read_csv(mainpath+r'train_queries.csv')
test_queries = pd.read_csv(mainpath+r'test_queries.csv')

In [4]:
train_queries.head()

,sid,pid,req_time,o,d
0,3000821,NaN,2018-11-02 17:54:30,"116.29,39.97","116.32,39.96"
1,3085857,210736.0,2018-11-16 10:53:10,"116.39,39.84","116.33,39.79"
2,2944522,NaN,2018-10-06 10:33:58,"116.31,39.93","116.27,40.00"
3,559931,202427.0,2018-11-23 14:54:11,"116.27,39.88","116.39,39.90"
4,2819352,172251.0,2018-10-30 11:48:41,"116.34,39.96","116.37,39.86"


In [5]:
test_queries.head()

,sid,pid,req_time,o,d
0,1126541,178395.0,2018-12-05 14:51:43,"116.41,39.92","116.46,39.95"
1,453685,179178.0,2018-12-01 12:50:33,"116.39,39.90","116.41,39.91"
2,2066073,184748.0,2018-12-07 10:17:35,"116.37,39.71","116.31,39.89"
3,452746,162795.0,2018-12-01 07:52:44,"116.33,39.97","116.31,39.98"
4,1431672,116384.0,2018-12-06 10:36:30,"116.51,39.68","116.51,39.68"


In [6]:
train_od = train_queries[['o','d']]
test_od = test_queries[['o','d']]

od_df = pd.concat([train_od,test_od]).drop_duplicates()

o_df = od_df[['o']]
d_df = od_df[['d']]

o_df.columns=['co']
d_df.columns=['co']


all_co = pd.concat([o_df,d_df]).drop_duplicates()

In [7]:
all_co.shape

(6525, 1)

In [8]:
all_co['lng'] = all_co['co'].apply(lambda x: float(x.split(',')[0]))
all_co['lat'] = all_co['co'].apply(lambda x: float(x.split(',')[1]))

In [10]:
cluster_df = all_co.copy()
cluster_df.head()

,co,lng,lat
0,"116.29,39.97",116.29,39.97
1,"116.39,39.84",116.39,39.84
2,"116.31,39.93",116.31,39.93
3,"116.27,39.88",116.27,39.88
4,"116.34,39.96",116.34,39.96


In [17]:
lng_mean = cluster_df['lng'].mean()
lat_mean = cluster_df['lat'].mean()

lng_mode = cluster_df['lng'].mode()[0]
lat_mode = cluster_df['lat'].mode()[0]

In [18]:
cluster_df['main_centroid_mean_dis']=abs(cluster_df['lng']-lng_mean)+abs(cluster_df['lat']-lat_mean)
cluster_df['main_centroid_mode_dis']=abs(cluster_df['lng']-lng_mode)+abs(cluster_df['lat']-lat_mode)

cluster_df.head()

,co,lng,lat,main_centroid_mean_dis,main_centroid_mode_dis
0,"116.29,39.97",116.29,39.97,0.173152,0.56
1,"116.39,39.84",116.39,39.84,0.203152,0.59
2,"116.31,39.93",116.31,39.93,0.193152,0.58
3,"116.27,39.88",116.27,39.88,0.283152,0.67
4,"116.34,39.96",116.34,39.96,0.133152,0.52
